# 📌 Résumé du projet
Dans le cadre de l’appel à manifestation d’intérêt de l’ADEME, notre équipe CesiCDP développe une solution intelligente visant à optimiser les tournées de livraison de biens ou services dans un environnement urbain complexe. L’objectif est de réduire les déplacements et la consommation énergétique tout en prenant en compte des contraintes réalistes et dynamiques du terrain : routes fermées, ralenties, ou évoluant dans le temps.

Nous modélisons ce problème sous forme d’un graphe pondéré représentant un réseau routier. Notre approche repose sur une méthode approchée, capable de s’adapter à des situations dynamiques et incomplètes, et d’obtenir de bonnes solutions rapidement sans garantie d’optimalité.

# 🧭 Introduction
Dans ce notebook, nous analysons et testons l'efficacité de notre algorithme de Recuit Simulé (Simulated Annealing) appliqué à l'optimisation des tournées de livraison.

L'objectif est de :
- Évaluer les performances de l'algorithme en termes de qualité des solutions et de temps d'exécution.
- Étudier la complexité algorithmique et spatiale.
- Comparer les résultats obtenus avec d'autres approches.

# ⚠️ Contraintes
Et les contraintes sont : 
- Routes dynamiques ou perturbations : Simuler des changements dynamiques dans les coûts ou la disponibilité des routes pendant la résolution.
- Utilisation de plusieurs véhicules: Il peut y avoir plusieurs sous-tournées plutôt qu'une seule grande.
- Coût ou restriction de passage sur certaines arêtes : Certaines routes peuvent être plus coûteuses ou interdites (par exemple, travaux ou routes bloquées).
- Ratio d'embouteillage moyen : Les données de Bison Futé sont utilisées pour calculer un ratio d'embouteillage moyen en fonction de l'heure.
- Capacités du véhicule : Chaque véhicule a une capacité limite pour transporter des marchandises ou des passagers.


# 🧱 Structure du graphe
Chaque ville est un nœud, chaque route une arête pondérée. Le graphe peut être enrichi par des données géographiques.
```python
from graph import Graph
graph = Graph.load('./data/datasets/size_200/graph_size200_density0.01.pkl')
graph.plot()
```


# 🔄 Pseudo-code de l'algorithme de Recuit Simulé
```plaintext
1. Initialiser une solution aléatoire (tournées par véhicule)
2. Répéter :
    a. Modifier dynamiquement le graphe
    b. Générer une solution voisine
    c. Accepter ou non selon la température
    d. Réduire la température
3. Retourner la meilleure solution
```
![Flowchart](flowchart_simulated_annealing.svg)


# 🧮 Analyse de la complexité spatiale
- Espace requis : O(n + m + k) où :
  - n : nombre de villes
  - m : nombre d'arêtes
  - k : nombre de véhicules


# 🧮 Choix des paramètres (mettre logo paramètres)
Utiliser librairie octuna

# 🧪 Test de performance
```python
# Exemple de test sur différents graphes
from tests import run_experiments
results = run_experiments()
results.head()
```


# ⚖️ Comparaison avec d'autres algorithmes
Comparaison avec une heuristique gloutonne sur les mêmes instances :
```python
from compare import compare_algorithms
compare_algorithms(graph)
```


# 📽️ Rendu visuel avec GIF
```python
from visual import render_solution_gif
render_solution_gif(best_solution, './animations/solution.gif')
```
![GIF](./animations/solution.gif)


# ✅ Conclusion
- Le recuit simulé est efficace sur les graphes denses.
- Robuste face aux perturbations.
- Temps raisonnables et résultats compétitifs.
- Possibilités d'amélioration avec des hybrides heuristiques.
